In [1]:
import pandas as pd
import numpy as np
from bayes_opt import BayesianOptimization
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [2]:
# def bayesian_optimization(dataset, function, parameters):
#    X_train, y_train, X_test, y_test = dataset
#    n_iterations = 5
#    gp_params = {"alpha": 1e-4}

#    BO = BayesianOptimization(function, parameters)
#    BO.maximize(n_iter=n_iterations, **gp_params)

#    return BO.max

In [3]:
# def rfc_optimization(cv_splits):
#     def function(n_estimators, max_depth, min_samples_split):
#         return cross_val_score(
#                RandomForestClassifier(
#                    n_estimators=int(max(n_estimators,0)),                                                               
#                    max_depth=int(max(max_depth,1)),
#                    min_samples_split=int(max(min_samples_split,2)), 
#                    n_jobs=-1, 
#                    random_state=42,   
#                    class_weight="balanced"),  
#                X=X_train, 
#                y=y_train, 
#                cv=cv_splits,
#                scoring="roc_auc",
#                n_jobs=-1).mean()

#     parameters = {"n_estimators": (10, 1000),
#                   "max_depth": (1, 150),
#                   "min_samples_split": (2, 10)}
    
#     return function, parameters

In [4]:
# def xgb_optimization(cv_splits, eval_set):
#     def function(eta, gamma, max_depth):
#             return cross_val_score(
#                    xgb.XGBClassifier(
#                        objective="binary:logistic",
#                        learning_rate=max(eta, 0),
#                        gamma=max(gamma, 0),
#                        max_depth=int(max_depth),                                               
#                        seed=42,
#                        nthread=-1,
#                        scale_pos_weight = len(y_train[y_train == 0])/
#                                           len(y_train[y_train == 1])),  
#                    X=X_train, 
#                    y=y_train, 
#                    cv=cv_splits,
#                    scoring="roc_auc",
#                    fit_params={
#                         "early_stopping_rounds": 10, 
#                         "eval_metric": "auc", 
#                         "eval_set": eval_set},
#                    n_jobs=-1).mean()

#     parameters = {"eta": (0.001, 0.4),
#                   "gamma": (0, 20),
#                   "max_depth": (1, 2000)}
    
#     return function, parameters

In [5]:
# def train(X_train, y_train, X_test, y_test, function, parameters):
#     dataset = (X_train, y_train, X_test, y_test)
#     cv_splits = 4
    
#     best_solution = bayesian_optimization(dataset, function, parameters)      
#     params = best_solution["params"]

#     model = RandomForestClassifier(
#              n_estimators=int(max(params["n_estimators"], 0)),
#              max_depth=int(max(params["max_depth"], 1)),
#              min_samples_split=int(max(params["min_samples_split"], 2)), 
#              n_jobs=-1, 
#              random_state=42,   
#              class_weight="balanced")

#     model.fit(X_train, y_train)
    
#     return model

In [6]:
import pandas as pd
import gc
import warnings

from bayes_opt import BayesianOptimization

from sklearn.model_selection import train_test_split
# from sklearn.cross_validation import cross_val_score, StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import log_loss, matthews_corrcoef, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import contextlib


# def read_dataset(fname):
#     data = pd.read_csv(fname, encoding="utf-8")
#     # drop掉无用数据
#     data.drop(['obs_time', 'patient_id', 'dataset_name'], axis=1, inplace=True)
#     return data

# fileName = open(r"/Users/tuyu/000000毕业设计/Flight_Delay/Train_Feature/final_feature.csv")
# data_file = pd.read_csv(fileName)
feature = pd.read_table('/Users/tuyu/000000毕业设计/Flight_Delay/Train_Feature/final_feature.csv',sep=',',encoding='gb2312')
del(feature['本时段离港平均延误时长'])
del(feature['上一时段离港平均延误时长'])
del(feature['航班编号'])
# 1.前序延误 = 前序航班 实际到达时间 - 计划到达时间
# 2.起飞间隔 = 当前航班的计划起飞时间 - 前序航班实际到达时间
feature['起飞间隔'] = feature['起飞间隔'].fillna(0)
feature['前序延误'] = feature['前序延误'].fillna(0)
feature['出发机场天气得分'] = feature['出发机场天气得分'].interpolate()
feature['到达机场天气得分'] = feature['到达机场天气得分'].interpolate()
feature['平均延误时间'] = feature['平均延误时间'].interpolate()
feature['最大延误时间'] = feature['最大延误时间'].interpolate()
feature['延误时间中位数'] = feature['延误时间中位数'].interpolate()
feature['延误时间标准差'] = feature['延误时间标准差'].interpolate()

feature['本时段计划离港航班数'] = feature['本时段计划离港航班数'].fillna(feature['本时段计划离港航班数'].mean())
feature['本时段实际离港航班数'] = feature['本时段实际离港航班数'].fillna(feature['本时段实际离港航班数'].mean())
feature['本时段进港延误时长'] = feature['本时段进港延误时长'].fillna(feature['本时段进港延误时长'].mean())
feature['本时段实际进港延误航班数'] = feature['本时段实际进港延误航班数'].fillna(feature['本时段实际进港延误航班数'].mean())
feature['本时段离港延误时长'] = feature['本时段离港延误时长'].fillna(feature['本时段离港延误时长'].mean())
feature['本时段离港延误率'] = feature['本时段离港延误率'].fillna(feature['本时段离港延误率'].mean())
feature['上一时段进港延误时长'] = feature['上一时段进港延误时长'].fillna(feature['上一时段进港延误时长'].mean())
feature['上一时段离港延误时长'] = feature['上一时段离港延误时长'].fillna(feature['上一时段离港延误时长'].mean())
feature['上一时段实际进港航班数'] = feature['上一时段实际进港航班数'].fillna(feature['上一时段实际进港航班数'].mean())
feature['上一时段进港延误率'] = feature['上一时段进港延误率'].fillna(feature['上一时段进港延误率'].mean())





In [7]:
feature0 = feature[feature['标签'] == 0]
feature0new = feature0.sample(frac=0.1, replace=False, random_state=None,axis=0)
feature1 = feature[feature['标签'] == 1]


feature0Bayes = feature0new.sample(frac=0.08, replace=False, random_state=None,axis=0)
feature1Bayes = feature1.sample(frac=0.06, replace=False, random_state=None,axis=0)

featureBayes = pd.concat([feature0Bayes,feature1Bayes],axis=0, ignore_index=True)

train = featureBayes
# # train = read_dataset('190624_data.csv')
y = train['标签'].values
X = train.drop(['标签'], axis=1).values

# # y = y + 0  # 将布尔值替换成01

In [8]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
X = min_max_scaler.fit_transform(X)
cols = train.columns

# 把数据的80%用来训练模型,20%做模型测试和评估,此处用到训练集-验证集二划分
p = 0.8  # 设置训练数据比例,
X_train = X[:int(len(X) * p), :]  # 前80%为训练集
X_test = X[int(len(X) * p):, :]  # 后20%为测试集
y_train = y[:int(len(y) * p)]  # 前80%为训练集
y_test = y[int(len(y) * p):]  # 后20%为测试集


# Comment out any parameter you don't want to test
def XGB_CV(
        learning_rate,
        max_depth,
        gamma,
        min_child_weight,
        max_delta_step,
        subsample,
        colsample_bytree
):
    global AUCbest
    global ITERbest

    # Define all XGboost parameters
    paramt = {
        'learning_rate': learning_rate,
        'booster': 'gbtree',
        'max_depth': int(max_depth),
        'gamma': gamma,
        'eta': 0.1,
        'objective': 'binary:logistic',
        'nthread': 4,
        'silent': True,
        'eval_metric': 'auc',
        'subsample': max(min(subsample, 1), 0),
        'colsample_bytree': max(min(colsample_bytree, 1), 0),
        'min_child_weight': min_child_weight,
        'max_delta_step': int(max_delta_step),
        'seed': 1001
    }

    folds = 5
    cv_score = 0

    print("\n Search parameters (%d-fold validation):\n %s" % (folds, paramt), file=log_file)
    log_file.flush()

    xgbc = xgb.cv(
        paramt,
        dtrain,
        num_boost_round=20000,
        stratified=True,
        nfold=folds,
        #                    verbose_eval = 10,
        early_stopping_rounds=100,
        metrics='auc',
        show_stdv=True
    )

    val_score = xgbc['test-auc-mean'].iloc[-1]
    train_score = xgbc['train-auc-mean'].iloc[-1]
    print(
        ' Stopped after %d iterations with train-auc = %f val-auc = %f ( diff = %f ) train-gini = %f val-gini = %f' % (
            len(xgbc), train_score, val_score, (train_score - val_score), (train_score * 2 - 1), (val_score * 2 - 1)))
    if (val_score > AUCbest):
        AUCbest = val_score
        ITERbest = len(xgbc)

    return (val_score * 2) - 1


log_file = open('Porto-AUC-5fold-XGB-run-01-v1-full.log', 'a')
AUCbest = -1.
ITERbest = 0
dtrain = xgb.DMatrix(X_train, label=y_train)

XGB_BO = BayesianOptimization(XGB_CV, {
    'learning_rate': (0.01, 1),
    'max_depth': (5, 12),
    'gamma': (0.001, 10.0),
    'min_child_weight': (0, 20),
    'max_delta_step': (0, 10),
    'subsample': (0.4, 1.0),
    'colsample_bytree': (0.4, 1.0)
})

# XGB_BO.explore({
#     'learning_rate': [0.01, 0.03, 0.01, 0.03, 0.1, 0.3, 0.1, 0.3],
#     'max_depth': [3, 8, 3, 8, 8, 3, 8, 3],
#     'gamma': [0.5, 8, 0.2, 9, 0.5, 8, 0.2, 9],
#     'min_child_weight': [0.2, 0.2, 0.2, 0.2, 12, 12, 12, 12],
#     'max_delta_step': [1, 2, 2, 1, 2, 1, 1, 2],
#     'subsample': [0.6, 0.8, 0.6, 0.8, 0.6, 0.8, 0.6, 0.8],
#     'colsample_bytree': [0.6, 0.8, 0.6, 0.8, 0.6, 0.8, 0.6, 0.8],
# })

print('-' * 130)
print('-' * 130, file=log_file)
log_file.flush()

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    XGB_BO.maximize(init_points=2, n_iter=10, acq='ei', xi=0.0)

print('-' * 130)
print('Final Results')
print(XGB_BO.res)
# print('Maximum XGBOOST value: %f' % XGB_BO.res['max']['max_val'])
# print('Best XGBOOST parameters: ', XGB_BO.res['max']['max_params'])
# print('-' * 130, file=log_file)
# print('Final Result:', file=log_file)
# print('Maximum XGBOOST value: %f' % XGB_BO.res['max']['max_val'], file=log_file)
# print('Best XGBOOST parameters: ', XGB_BO.res['max']['max_params'], file=log_file)
# log_file.flush()
# log_file.close()

# history_df = pd.DataFrame(XGB_BO.res['all']['params'])
# history_df2 = pd.DataFrame(XGB_BO.res['all']['values'])
# history_df = pd.concat((history_df, history_df2), axis=1)
# history_df.rename(columns={0: 'gini'}, inplace=True)
# history_df['AUC'] = (history_df['gini'] + 1) / 2
# history_df.to_csv('Porto-AUC-5fold-XGB-run-01-v1-grid.csv')

----------------------------------------------------------------------------------------------------------------------------------
|   iter    |  target   | colsam... |   gamma   | learni... | max_de... | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------------------------------
 Stopped after 66 iterations with train-auc = 0.977102 val-auc = 0.949101 ( diff = 0.028001 ) train-gini = 0.954205 val-gini = 0.898202
|  1        |  0.8982   |  0.4702   |  6.089    |  0.2584   |  1.574    |  10.98    |  1.525    |  0.7647   |
 Stopped after 207 iterations with train-auc = 0.962166 val-auc = 0.946994 ( diff = 0.015171 ) train-gini = 0.924331 val-gini = 0.893988
|  2        |  0.894    |  0.5258   |  9.254    |  0.2114   |  1.033    |  8.538    |  17.31    |  0.4496   |
 Stopped after 1765 iterations with train-auc = 0.963177 val-auc = 0.949939 ( diff = 0.013237 ) train-gini = 0.926353 val-gini = 0.899878
|  3        |  0.8

In [15]:


from xgboost import XGBClassifier
# xgboost = XGBClassifier(
#  learning_rate =0.01754,
#  n_estimators=1237 ,
#  max_depth=11,
#     eta = 0.1,
#  min_child_weight=15.43,
#  gamma=0.2631,
#  subsample=0.73348,
#  colsample_bytree=0.4334,
#  max_delta_step=0.5662,
#  objective= 'binary:logistic',
#  nthread=4,
#  seed=1001)

#         'learning_rate': learning_rate,
#         'booster': 'gbtree',
#         'max_depth': int(max_depth),
#         'gamma': gamma,
#         'eta': 0.1,
#         'objective': 'binary:logistic',
#         'nthread': 4,
#         'silent': True,
#         'eval_metric': 'auc',
#         'subsample': max(min(subsample, 1), 0),
#         'colsample_bytree': max(min(colsample_bytree, 1), 0),
#         'min_child_weight': min_child_weight,
#         'max_delta_step': int(max_delta_step),
#         'seed': 1001

xgboost = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 seed=27)


xgboost_model = xgboost.fit(X_train, y_train.astype('int'))


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [ ]:
#XGBOOST模型调参,测试样本0.1，训练集占0.9，模型效果为：
# XGBOOST模型评估
# 训练样本的准确率: 0.9718
# average_precision: 0.6414
# 查准率: 0.8901
# 召回率: 0.6960
# F1-Score: 0.7811
# AUC-Score_predprob: 0.9639
# 验证样本的准确率: 0.9710
# 查准率: 0.8856
# 召回率: 0.6887
# F1-Score: 0.7748
# AUC-Score: 0.9609
# ('ks', 0.6817358342785097)

In [ ]:
import pickle
f= open('/Users/tuyu/000000毕业设计/Flight_Delay/Model/model/XGBOOST_Bayes_model.pickle','wb')
pickle.dump(xgboost_model,f)
f.close()

# 看上面结果，迭代了12次，train-auc和val-auc，选取最好的，对应下面的参数，参数也有12组